In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# The current version of seaborn generates a bunch of warnings that we'll ignore
import warnings 
warnings.filterwarnings("ignore")
sns.set_style('darkgrid')
import os
import time
from datetime import datetime
#!pip install pendulum
#!pip install pyramid-arima
import pendulum
#from pyramid.arima import auto_arima

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb 
from sklearn.ensemble import GradientBoostingRegressor 
from xgboost import XGBRegressor
 
#Directory needs to be changed 
DIR = '/content/gdrive/My Drive/Sales Forecasting/' 

In [0]:
# Mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def getData(folder_path, file_name):
    ext = file_name.split(".")[-1].lower()
    if ext == "csv":
        data = pd.read_csv(os.path.join(folder_path, file_name))
    elif ext == "xlsx":
        data = pd.read_excel(os.path.join(folder_path, file_name))
    return data

In [0]:
#Load the dataset
sales_train = getData(DIR, 'train.csv')
features = getData(DIR, 'features.csv')
stores = getData(DIR, 'stores.csv')
sales_test = getData(DIR, 'test.csv')

# 1) Data Cleaning

*   In time independent data (non-time-series), a common practice is to fill the gaps with the mean or median value of the field. However, this is not applicable in the time series. Missing values in columns "CPI" & "Unemployment" can be imputed using rolling average, interpolation techniques (linear, time, spline) & ARIMA. In this notebook, we use rolling average to impute missing values.

*   Weekly sales column has negative values. We assume that this column can’t be negative, and hence convert them to positives.

```
# Using ARIMA to impute missing values.
stores = list(range(1,46))
for store in stores:
  temp_df = features[features['Store'] == store][['Date','CPI']]
  temp_df.Date = pd.to_datetime(temp_df.Date)
  temp_df = temp_df.set_index('Date')
    
  # The problem with plain ARIMA model is that it does not account for seasonality.
  # If the time series exhibits seasonality, then, SARIMA is better as it uses seasonal differencing.
  ts_model = auto_arima(temp_df[:169], 
                            start_p=0, start_q=0, start_P=0, start_Q=0,
                            max_p=3, max_q=3, # maximum p and q
                            max_P=3, max_Q=3, # maximum P and Q
                            m=12, 
                            seasonal=True, # No seasonality 
                            test='adf', # use Augmented Dickey-Fuller(adftest) test to find optimal 'd'
                            seasonal_test='ch', # use CHTest to find optimal 'D'
                            d=None, # let model determine 'd' 
                            D=None, # let model determine 'D' 
                            trace=True, error_action='ignore', suppress_warnings=True, stepwise=True)

  future_forecast = ts_model.predict(n_periods=13)

  features.loc[(features.Store == store) & (features.CPI.isnull()),'CPI'] = future_forecast
  ```



In [0]:
# Using rolling average to fill the missing values in CPI and Unemployment columns
stores_list = list(range(1,46))
for store in stores_list:
  temp_df = features[features['Store'] == store]
  temp_df.Date = pd.to_datetime(temp_df.Date)
  temp_df = temp_df.set_index('Date')
  
  features.loc[(features.Store == store),['CPI']] = list(temp_df.CPI.fillna(temp_df.CPI.rolling(14,min_periods=1).mean()))
  features.loc[(features.Store == store),['Unemployment']] = list(temp_df.Unemployment.fillna(temp_df.Unemployment.rolling(14,min_periods=1).mean()))

In [0]:
sales_train['Weekly_Sales'] = sales_train['Weekly_Sales'].abs() 

# 2) Feature Engineering


*   Since we are going to fit tree-based models, feature scaling is not required. Random Forest/Boosted trees use information gain/gini coefficient inherently which will not be affected by scaling unlike many other machine learning models (such as Linear Regression, PCA)

*   We One hot encode "Type" categorical feature since it contains more than 2 levels (A/B/C)

*   We label encode the "IsHoliday" categorical feature since it contains just 2 levels (True/False)

*   We create new features namely - "IsSuperbowl", "IsLaborday", "IsThanksgiving", "IsChristmas" to identify the type of holiday 

*   Extract new features from "Date" feature such as year, month, week of year, week of month, day etc

*   Each of the feature "Markdown1", "Markdown2", "Markdown3", "Markdown4", and "Markdown5" has around 65% missing date, we ignore these variables from our analysis.




## 2.1) Merge and join dataframes

In [0]:
temp1=pd.merge(sales_train, features, how='left')
train=pd.merge(temp1, stores, how='left')

In [0]:
temp2=pd.merge(sales_test, features, how='left')
test=pd.merge(temp2, stores, how='left')

In [0]:
# Concatenating train & test
train['train_or_test'] = 'train'
test['train_or_test'] = 'test'
merged_df = pd.concat([train,test], sort=False)
print('Combined merged_df shape:{}'.format(merged_df.shape))

Combined merged_df shape:(536634, 17)


In [0]:
# Sorting the dataframe by store then Dept then date
merged_df = merged_df.sort_values(by=['Store','Dept','Date'], axis=0).reset_index().drop(columns='index')

## 2.2) Categorical Encodings

In [0]:
# One-hot-encode "Type" categorical variables 
merged_df = pd.get_dummies(merged_df, columns=['Type'])

In [0]:
# Label-encode "IsHoliday" categorical variables 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
merged_df['IsHoliday'] = le.fit_transform(merged_df['IsHoliday'])

In [0]:
# Create new features namely - "IsSuperbowl", "IsLaborday", "IsThanksgiving", "IsChristmas" 
def IsSuperbowl(x):
  if (x == '2010-02-12') | (x == '2011-02-11') | (x == '2012-02-10') | (x == '2013-02-08'):
    return 1
  else:
    return 0

def IsLaborday(x):
  if (x == '2010-09-10') | (x == '2011-09-09') | (x == '2012-09-07') | (x == '2013-09-06'):
    return 1
  else:
    return 0

def IsThanksgiving(x):
  if (x == '2010-11-26') | (x == '2011-11-25') | (x == '2012-11-23') | (x == '2013-11-29'):
    return 1
  else:
    return 0

def IsChristmas(x):
  if (x == '2010-12-31') | (x == '2011-12-30') | (x == '2012-12-28') | (x == '2013-12-27'):
    return 1
  else:
    return 0

merged_df['IsSuperbowl'] = merged_df['Date'].apply(lambda x: IsSuperbowl(x))
merged_df['IsLaborday'] = merged_df['Date'].apply(lambda x: IsLaborday(x))
merged_df['IsThanksgiving'] = merged_df['Date'].apply(lambda x: IsThanksgiving(x))
merged_df['IsChristmas'] = merged_df['Date'].apply(lambda x: IsChristmas(x))

## 2.3) Datetime Features

In [0]:
# Extract features from "Date" feature
merged_df['WeekofMonth'] = merged_df['Date'].apply(lambda x: pendulum.parse(x).week_of_month)
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Year'] = merged_df['Date'].dt.year
merged_df['Month'] = merged_df['Date'].dt.month
merged_df['Week'] = merged_df['Date'].dt.week
merged_df['Day'] = merged_df['Date'].dt.day

In [0]:
# Drop features namely "Markdown1", "Markdown2", "Markdown3", "Markdown4", and "Markdown5"
merged_df = merged_df.drop(['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5'], axis=1)

## 2.4) Featuretools




```
'''
# https://towardsdatascience.com/automated-feature-engineering-in-python-99baf11cc219
# https://towardsdatascience.com/the-hitchhikers-guide-to-feature-extraction-b4c157e96631
# https://mlwhiz.com/blog/2019/05/19/feature_extraction/
# https://medium.com/@rrfd/simple-automatic-feature-engineering-using-featuretools-in-python-for-classification-b1308040e183
# https://www.analyticsvidhya.com/blog/2018/08/guide-automated-feature-engineering-featuretools-python/

import featuretools as ft

# Create new entityset
es = ft.EntitySet(id = 'forecasting')

# Create an entity from the "Sales" dataframe
sales_train_ft = sales_train.copy()
sales_train_ft["index"] = sales_train_ft["Store"].astype(str) + "_" + sales_train_ft["Dept"].astype(str) + "_" + sales_train_ft["Date"].astype(str)
es = es.entity_from_dataframe(entity_id = 'sales', dataframe = sales_train_ft,
                              index = 'index', variable_types = {"Date": ft.variable_types.Datetime, "IsHoliday": ft.variable_types.Categorical})

# Create an entity from the "Features" dataframe
store_dept_ft = pd.DataFrame(sales_train_ft[['Store',	'Dept', 'Date']].drop_duplicates().values.tolist(), columns = ['Store', 'Dept', 'Date'])
features_ft = features.copy()
features_ft = pd.merge(store_dept_ft, features_ft, on=['Store', 'Date'], how='left')
features_ft["index"] = features_ft["Store"].astype(str) + "_" + features_ft["Dept"].astype(str) + "_" + features_ft["Date"].astype(str)
es = es.entity_from_dataframe(entity_id="features",
                                 dataframe=features_ft,
                                 index="index",
                               variable_types={"Date": ft.variable_types.Datetime})

# Create an entity from the "Stores" dataframe
stores_ft = stores.copy()
stores_ft = pd.merge(stores_ft, sales_train_ft[['Store','Dept','Date']], on='Store', how='left')
stores_ft["index"] = stores_ft["Store"].astype(str) + "_" + stores_ft["Dept"].astype(str) + "_" + stores_ft["Date"].astype(str)
es = es.entity_from_dataframe(entity_id="stores",
            dataframe=stores_ft,
            index="index",
            variable_types={"Type": ft.variable_types.Categorical, "Date": ft.variable_types.Datetime})

# adding the relationship
relationship1 = ft.Relationship(es["sales"]["index"],
                       es["features"]["index"])

# Add the relationship to the entity set
es = es.add_relationship(relationship1)

# adding the relationship
relationship2 = ft.Relationship(es["sales"]["index"],
                       es["stores"]["index"])

# Add the relationship to the entity set
es = es.add_relationship(relationship2)

#create features
feature_matrix, feature_defs = ft.dfs(entityset=es, target_entity="sales",max_depth = 2)```



## 2.5) Previous sales values  

In [0]:
# Features constructed from previous sales values
# Create various sales agg features with given agg functions
def create_sales_agg_monthwise_features(df, gpby_cols, target_col, agg_funcs):
    gpby = df.groupby(gpby_cols)
    newdf = df[gpby_cols].drop_duplicates().reset_index(drop=True)
    for agg_name, agg_func in agg_funcs.items():
        aggdf = gpby[target_col].agg(agg_func).reset_index()
        aggdf.rename(columns={target_col:'Monthly_Sales_'+agg_name}, inplace=True)
        newdf = newdf.merge(aggdf, on=gpby_cols, how='left')
    return newdf

# Creating sales lag features
def create_sales_lag_feats(df, gpby_cols, target_col, lags):
    gpby = df.groupby(gpby_cols)
    for i in lags:
        df['_'.join([target_col, 'lag', str(i)])] = gpby[target_col].shift(i).values 
    return df

# Creating sales exponentially weighted mean features
def create_sales_ewm_feats(df, gpby_cols, target_col, alpha=[0.9], shift=[1]):
    gpby = df.groupby(gpby_cols)
    for a in alpha:
        for s in shift:
            df['_'.join([target_col, 'lag', str(s), 'ewm', str(a)])] = gpby[target_col].apply(lambda x: x.shift(52).ewm(alpha=0.95).mean())
    return df

In [0]:
# creating sales lag features 
merged_df_new = create_sales_lag_feats(merged_df, gpby_cols=['Store','Dept'], 
                                       target_col='Weekly_Sales', lags=[52])

# creating ewm features 
merged_df_new = create_sales_ewm_feats(merged_df_new, gpby_cols=['Store','Dept'], 
                                       target_col='Weekly_Sales', alpha=[0.95, 0.9, 0.8, 0.7, 0.6, 0.5],
                                       shift=[52])

# creating sales monthwise aggregated values
agg_df = create_sales_agg_monthwise_features(merged_df_new.loc[~(merged_df_new.train_or_test=='test'), :], gpby_cols=['Store','Dept', 'Month'], 
                                               target_col='Weekly_Sales', 
                                             agg_funcs={'mean':np.mean, 
                                              'median':np.median, 'max':np.max, 
                                              'min':np.min, 'std':np.std})
# # Joining agg_df with merged_df_new
merged_df_new = merged_df_new.merge(agg_df, on=['Store','Dept', 'Month'], how='left')

In [0]:
# Final train and test datasets
train = merged_df_new.loc[merged_df_new.train_or_test=='train', :]
test = merged_df_new.loc[merged_df_new.train_or_test=='test', :]
print('Train shape:{}, Test shape:{}'.format(train.shape, test.shape))

Train shape:(421570, 35), Test shape:(115064, 35)


In [0]:
# merged_df_new.to_csv(os.path.join(DIR, 'Check.csv'))
# Lagged columns in the train data contains NaN's and we remove those rows
train = train.drop(['Date', 'train_or_test'], axis=1).dropna() 

# Lagged columns in the test data contains NaN's because of not being able to find previous years sales 
# Hence we will fill them with 0's
test = test.drop(['Date', 'train_or_test', 'Weekly_Sales'], axis=1) .fillna(0)

# 3) Machine Learning Modelling




*   If we optimize the model for the training data, then our model will score very well on the training set, but will not be able to generalize to new data, such as in a test set. When a model performs highly on the training set but poorly on the test set, this is known as overfitting, or essentially creating a model that knows the training set very well but cannot be applied to new problems. An overfit model may look impressive on the training set, but will be useless in a real application. Therefore, we tune hyperparameters cross validation.



*   Using Scikit-Learn’s RandomizedSearchCV method, we can define a grid of hyperparameter ranges, and randomly sample from the grid, performing K-Fold CV with each combination of values. The most important arguments in RandomizedSearchCV are n_iter, which controls the number of different combinations to try, and cv which is the number of folds to use for cross validation (we use 100 and 3 respectively). More iterations will cover a wider search space and more cv folds reduces the chances of overfitting, but raising each will increase the run time. Machine learning is a field of trade-offs, and performance vs time is one of the most fundamental.




## 3.1) By Store & Department combination

*   The following store-department combos are present in test set but not in trian set: 

5-99, 9-99, 10-99, 18-43, 24-43, 25-99, 34-39, 36-30, 37-29, 42-30, 45-39




In [0]:
# Store parameters for regressors
# Random Forest parameters
rf_params =  {'n_estimators': [100],     # number of trees in the forest
              'max_features': ['auto'], # number of features to consider when looking for the best split
              'max_depth': [10, 15],        # maximum number of levels in tree
              'min_samples_split': [2, 4], # minimum number of samples required to split a node
              'min_samples_leaf': [2]} # minimum number of samples required at each leaf node
              #'bootstrap': [True, False]} # method for sampling data points (with or without replacement)

# Light GBM parameters
lgbm_params = {'boosting_type': ['gbdt'],
                'num_leaves': [10, 15], # number of leaves in full tree
                'max_depth': [5, 10, 15],  # maximum depth of tree. This parameter is used to handle model overfitting. 
                'learning_rate': [0.001, 0.01, 0.1], # determines the impact of each tree on the final outcome. 
                                        # LGBM works by starting with an initial estimate which is updated using the output of each tree. 
                                        # learning parameter controls the magnitude of this change in the estimates. Typical values: 0.1, 0.001, 0.003
                'n_estimators': [100, 150], # Number of boosted trees to fit.
                'objective': ['regression']}

# Gradient Boosting parameters
gbm_params = {'max_depth': [2],
              'n_estimators': [3],
              'learning_rate': [0.1]}

# Extreme Gradient Boosting parameters
xgb_params = {'learning_rate': [0.01, 0.1, 0.15],
              'max_depth': [4, 5, 6, 8],
              'subsample': [0.8, 0.9],
              'gamma': [1],
              'colsample_bytree': [0.9, 1]}


In [0]:
def write_csv(pred_values, file_name):
  kaggle_check = pd.DataFrame()
  kaggle_check['Id'] = sales_test['Store'].astype(str) + '_' +  sales_test['Dept'].astype(str) + '_' +  sales_test['Date'].astype(str)
  kaggle_check['Weekly_Sales'] = pred_values
  return kaggle_check.to_csv(os.path.join(DIR, file_name), index=False)

In [0]:
# Class to fit models by store+dept combo
class TrainByStoreDeptCombo(object):
    def __init__(self, clf, seed=0, params=None):
        self.clf = clf()
        self.params = params

    def train_predict(self, train, test):

        pred_values = []
        # this model is trained on store+dept filtered data
        store_dept_list = test[['Store',	'Dept']].drop_duplicates().values.tolist()

        for tempStore, tempDept in store_dept_list:

          print ('Store No: ', tempStore, ' Dept No:', tempDept)
          x_train = train[(train['Store'] == tempStore) & (train['Dept'] == tempDept)].drop(['Weekly_Sales'], axis=1)
          y_train = train[(train['Store'] == tempStore) & (train['Dept'] == tempDept)]['Weekly_Sales']
          x_test = test[(test['Store'] == tempStore) & (test['Dept'] == tempDept)]
        
          # sample size restrictions since RandomizedSearchCV fails if there isn't enough data
          # we cannot have number of splits (cv=5) greater than the number of samples. we need a minimum of 5 training samples for a 5-fold cross validation
          # hence the first condition: if training samples for store+dept combo is >= 5, we perform cross validation
          # if training samples for store+dept combo is >= 1 and < 5, we do not perform cross validation
          # but if training samples for store+dept combo is 0, we predict the sales to be average sales for the department  
          if len(x_train) >= 5:
          
            # random search of parameters, using 5 fold cross validation
            rgr = RandomizedSearchCV(estimator=self.clf, param_distributions = self.params, scoring='neg_root_mean_squared_error', 
                                     iid=False, n_jobs=2, n_iter = 10, cv=5, verbose=5)
            rgr = rgr.fit(x_train, y_train)

            # validate the best model with optimized number of estimators
            rgr = rgr.best_estimator_.fit(x_train, y_train)

            # predict values
            predict_test = rgr.predict(x_test)

          elif len(x_train) >= 1: 
            # fit model
            rgr = self.clf.fit(x_train, y_train)

            # predict values
            predict_test = rgr.predict(x_test)

          else:
            predict_test = np.repeat(np.average(sales_train[sales_train['Dept'] == tempDept]['Weekly_Sales']), len(x_test))
          
          # store the predicted values
          pred_values.extend(predict_test)

        return pred_values

In [0]:
rf = TrainByStoreDeptCombo(clf=RandomForestRegressor, seed=0, params=rf_params)
rf_pred = rf.train_predict(train, test)
write_csv(rf_pred, 'output_StoreDept_rf.csv')

In [0]:
lgbm = TrainByStoreDeptCombo(clf=lgb.LGBMRegressor, seed=0, params=lgbm_params)
lgbm_pred = lgbm.train_predict(train, test)
write_csv(lgbm_pred, 'output_StoreDept_lgbm.csv')

In [0]:
gbm = TrainByStoreDeptCombo(clf=GradientBoostingRegressor, seed=0, params=gbm_params)
gbm_pred = gbm.train_predict(train, test)
write_csv(gbm_pred, 'output_StoreDept_gbm.csv')

## 3.2) By Department

In [0]:
# Class to fit models by department
class TrainByDept(object):
    def __init__(self, clf, seed=0, params=None):
        self.clf = clf()
        self.params = params

    def train_predict(self, train, test):

        pred_values = []
        dept_list = test['Dept'].drop_duplicates().values.tolist()

        for tempDept in dept_list:

          print ('Dept No:', tempDept)
          x_train = train[train['Dept'] == tempDept].drop(['Weekly_Sales'], axis=1)
          y_train = train[train['Dept'] == tempDept]['Weekly_Sales']
          x_test = test[test['Dept'] == tempDept]
        
          # there are enough number of training samples for each department and hence sample size restictions are not required 
          # RandomizedSearchCV can applied for each department 
          # grid search for best parameters
          if len(x_train) >= 5:

            rgr = RandomizedSearchCV(estimator=self.clf, param_distributions = self.params, scoring='neg_root_mean_squared_error', 
                                      iid=False, n_jobs=2, n_iter = 10, cv=5, verbose=5)
            
            rgr = rgr.fit(x_train, y_train)

            # validate the best model with optimized number of estimators
            rgr = rgr.best_estimator_.fit(x_train, y_train)

            # predict values
            predict_test = rgr.predict(x_test)           

          elif len(x_train) >= 1: 
            # fit model
            rgr = self.clf.fit(x_train, y_train)

            # predict values
            predict_test = rgr.predict(x_test)

          else:
            predict_test = np.repeat(np.average(sales_train[sales_train['Dept'] == tempDept]['Weekly_Sales']), len(x_test))

          # store the predicted values
          pred_values.extend(predict_test)

        return pred_values

In [0]:
rf = TrainByDept(clf=RandomForestRegressor, seed=0, params=rf_params)
rf_pred = rf.train_predict(train, test)
write_csv(rf_pred, 'output_dept_rf.csv')

# 4) Time Series Modeling


*   In total there are ~3300 store department combinations.

*   The training data has gaps in dates. For any store department combinations that has missing dates, I simply used 0 for weekly sales and make sure that any store department combinations has 143 weekly dates.


## 4.1) Prophet

In [0]:
from fbprophet import Prophet

init_ts = merged_df_new[['Store',	'Dept',	'Date',	'Weekly_Sales', 'train_or_test']]
init_ts = init_ts.set_index('Date')
ts_store_dept = init_ts[['Store', 'Dept']].drop_duplicates().reset_index(drop=True).values.tolist()

ts = []
idx = pd.date_range('2/5/2010', '10/26/2012', freq='W-FRI')

for store, dept in ts_store_dept:
  tmp_ts = init_ts[(init_ts['Store'] == store) & (init_ts['Dept'] == dept) & (init_ts['train_or_test'] == 'train')].reindex(idx)
  values = {'Store': store, 'Dept': dept, 'Weekly_Sales': 0, 'train_or_test': 'train'}
  tmp_ts = tmp_ts.fillna(value=values)
  
  ts.append(tmp_ts)
  ts.append(init_ts[(init_ts['Store'] == store) & (init_ts['Dept'] == dept) & (init_ts['train_or_test'] == 'test')])

ts = pd.concat(ts)

Create a holidays dataframe with two columns (holiday and ds) and a row for each occurrence of the holiday. It must include all occurrences of the holiday, both in the past (back as far as the historical data go) and in the future (out as far as the forecast is being made).

In [0]:
Superbowl = pd.DataFrame({
  'holiday' : 'superbowl',
  'ds' : pd.to_datetime(['2010-02-12','2011-02-11','2012-02-10','2013-02-08'])
  })

Laborday = pd.DataFrame({
  'holiday' : 'laborday',
  'ds' : pd.to_datetime(['2010-09-10','2011-09-09','2012-09-07','2013-09-06'])
  })

Thanksgiving = pd.DataFrame({
  'holiday' : 'thanksgiving',
  'ds' : pd.to_datetime(['2010-11-26','2011-11-25','2012-11-23','2013-11-29'])
  })

Christmas = pd.DataFrame({
  'holiday' : 'christmas',
  'ds' : pd.to_datetime(['2010-12-31','2011-12-30','2012-12-28','2013-12-27'])
  })

holidays = pd.concat((Superbowl, Laborday, Thanksgiving, Christmas))

In [0]:
ts_p = ts.copy()
ts_p = ts_p.reset_index().rename(columns = {'index':'Date'})
ts_forecast = []
store_dept = test[['Store',	'Dept']].drop_duplicates().values.tolist()

for store, dept in store_dept:
  
  print ('Store No: ', store, ' Dept No:', dept)

  # Lets narrow our analysis for a single store+dept combo 
  df = ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'train')][['Date', 'Weekly_Sales']]

  # Prophet only takes data as a dataframe with a ds (datestamp) and y (value we want to forecast) column. 
  df = df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})\

  if len(df) >= 5:
    # Create an instance of the Prophet class and then fit our dataframe to it.
    prophet = Prophet(holidays=holidays)
    prophet.fit(df)

    # Create a dataframe with the dates for which we want a prediction to be made with make_future_dataframe(). 
    # Then specify the number of days to forecast using the periods parameter.
    df_forecast = prophet.make_future_dataframe(periods=39, freq='W-FRI')

    # Call predict to make a prediction and store it in the forecast dataframe.
    df_forecast = prophet.predict(df_forecast)

    # Retrieve predictions for the dates in the test set
    df_forecast = pd.merge(ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'test')][['Store', 'Dept', 'Date']], 
                           df_forecast[['ds', 'yhat']], 
                           left_on='Date', right_on='ds', how = 'left')[['Store', 'Dept', 'Date', 'yhat']]

  else:

    forecast = np.repeat(np.average(ts_p[(ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'train')]['Weekly_Sales']), 
                         len(ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'test')]))

    df_forecast = ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'test')][['Store', 'Dept', 'Date']]

    df_forecast['yhat'] = forecast

  ts_forecast.append(df_forecast)

ts_forecast = pd.concat(ts_forecast)
ts_forecast['Store'] = ts_forecast['Store'].astype(int)
ts_forecast['Dept'] = ts_forecast['Dept'].astype(int)
ts_forecast['Id'] = ts_forecast['Store'].astype(str) + '_' +  ts_forecast['Dept'].astype(str) + '_' +  ts_forecast['Date'].astype(str)
ts_forecast = ts_forecast.rename(columns = {'yhat': 'Weekly_Sales'})
ts_forecast = ts_forecast[['Id', 'Weekly_Sales']]
ts_forecast.to_csv(os.path.join(DIR, 'Output_Prohpet.csv'), index=False)

## 4.2) ARIMA

In [0]:
import itertools
import statsmodels.api as sm

ts_arima = ts.copy()
ts_arima = ts_arima.reset_index().rename(columns = {'index':'Date'})
ts_forecast = []
store_dept_list = test[['Store',	'Dept']].drop_duplicates().values.tolist()

p = d = q = range(0, 2)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

for store, dept in store_dept_list:
  
  print ('Store No: ', store, ' Dept No:', dept)

  # Lets narrow our analysis for a single store+dept combo 
  df = ts_arima[(ts_arima['Store'] == store) &  (ts_arima['Dept'] == dept) & (ts_arima['train_or_test'] == 'train')][['Weekly_Sales']]

  if len(df) >= 5:
    # The problem with plain ARIMA model is that it does not account for seasonality.
    # If the time series exhibits seasonality, then, SARIMA is better as it uses seasonal differencing.
    # Create empty list to store search results
    order_aic_bic=[]

    for param in pdq:
      for param_seasonal in seasonal_pdq:
        try:
          # create and fit ARMA(p,q) model
          mod = sm.tsa.statespace.SARIMAX(df,
                                          order=param,
                                          seasonal_order=param_seasonal,
                                          enforce_stationarity=False,
                                          enforce_invertibility=False)
          results = mod.fit()
          print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))

          # Append order and results tuple
          order_aic_bic.append(((param,param_seasonal, results.aic, results.bic)))
        except:
          continue

    # Construct DataFrame from order_aic_bic
    order_df = pd.DataFrame(order_aic_bic, columns=['pdq', 'seasonal_pdq', 'AIC', 'BIC'])

    # Print order_df in order of increasing AIC
    #print(order_df.sort_values('AIC'))

    # Print order_df in order of increasing BIC
    # print(order_df.sort_values('BIC'))  

    opti_pdq = order_df.sort_values('AIC')['pdq'].iloc[0]
    opti_seasonal_pdq = order_df.sort_values('AIC')['seasonal_pdq'].iloc[0]

    # Instantiate the model
    model = sm.tsa.statespace.SARIMAX(df,
                                      order=opti_pdq,
                                      seasonal_order=opti_seasonal_pdq,
                                      enforce_stationarity=False,
                                      enforce_invertibility=False)

    # Fit the model
    results = model.fit()

    # Print model fit summary
    #print(results.summary())

    # Generate predictions
    forecast = results.get_forecast(steps=39)

    # Mean Forecast
    future_forecast = forecast.predicted_mean

    df_forecast = ts_arima[(ts_arima['Store'] == store) &  (ts_arima['Dept'] == dept) & (ts_arima['train_or_test'] == 'test')][['Store', 'Dept', 'Date']]

    df_forecast['yhat'] = future_forecast

  else:

    forecast = np.repeat(np.average(ts_p[(ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'train')]['Weekly_Sales']), 
                         len(ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'test')]))

    df_forecast = ts_p[(ts_p['Store'] == store) &  (ts_p['Dept'] == dept) & (ts_p['train_or_test'] == 'test')][['Store', 'Dept', 'Date']]

    df_forecast['yhat'] = forecast

  ts_forecast.append(df_forecast)

ts_forecast = pd.concat(ts_forecast)
ts_forecast['Store'] = ts_forecast['Store'].astype(int)
ts_forecast['Dept'] = ts_forecast['Dept'].astype(int)
ts_forecast['Id'] = ts_forecast['Store'].astype(str) + '_' +  ts_forecast['Dept'].astype(str) + '_' +  ts_forecast['Date'].astype(str)
ts_forecast = ts_forecast.rename(columns = {'yhat': 'Weekly_Sales'})
ts_forecast = ts_forecast[['Id', 'Weekly_Sales']]
ts_forecast.to_csv(os.path.join(DIR, 'output_sarima.csv'), index=False)

## 5) Averaged base models 

*   We just average two models here Random Forest, and Prohpet. 
*   Of course we could easily add more models in the mix.


In [0]:
def AveragingModels(*args):
  predictions = np.column_stack(args)
  return np.mean(predictions, axis=1)

In [0]:
rf_prohpet_pred = AveragingModels(rf_pred,prohpet_pred)

In [0]:
write_csv(rf_prohpet_pred, 'output_rf_prohpet.csv')